In [1]:
from captum.attr import ShapleyValueSampling
from tqdm import trange

from load_data import load_data
from train_models import *
from segmentation import *
from utils import *
import torch
import os

c:\Users\Nikos\miniconda3\envs\env_segment_shap\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# device for torch
from torch.cuda import is_available as is_GPU_available
device = "cuda" if is_GPU_available() else "cpu"

# dictionary mapping predictors to torch vs other, step necessary for Captum 
predictors = {
	'torch' : ['resNet'],
	'scikit' : ['miniRocket','randomForest']
}
segmentation_dict = {"clasp":get_claSP_segmentation, "infogain": get_InformationGain_segmentation, "greedygaussian": get_GreedyGaussian_segmentation, "equal": get_equal_segmentation, "nnsegment": get_NNSegment_segmentation}

In [3]:
# hyperparameters
n_background = 50

# settings
dataset_names = {'gunpoint'}
predictor_names = {"randomForest", 'miniRocket', 'resNet'}
segmentation_names = {"clasp", "infogain", "greedygaussian", "equal", "nnsegment"}
segmentation_types = {"default", "normalized"}
background_types = {"average", "zero", "sampling"}

# demo
# dataset_names = {'gunpoint'}
# predictor_names = {"randomForest"}
# segmentation_names = {"clasp"}
# segmentation_types = {"default", "normalized"}
# background_types = {"average", "zero"}

In [12]:
results_demo = {"dataset_name": 
                        {"segmentation_name": 
                                        {"predictor_name":{
                                                'attributions' : {
                                                        "background_name":{
                                                                "default": None, "normalized": None}
                                                },
                                                'y_test_pred' : None,        
                                        }, 
                                        "segments": None}, 
                        'y_test_true': None,
                        'label_mapping': None}
                        }

In [21]:
results = dict()

for dataset_name in dataset_names:
    # init dataset
    # load data
    X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name)
    # for debugging only
    # X_test = X_test[:2]
    # y_test = y_test[:2]
    n_samples, n_chs, ts_length = X_test.shape

    results[dataset_name] = {'y_test_true': y_test, 'label_mapping':enc}

    predictor_dict = dict()
    for predictor_name in predictor_names:

        if predictor_name=='resNet':
            clf,preds = train_ResNet(X_train, y_train, X_test, y_test, dataset_name,device=device)
        elif predictor_name=='miniRocket':
            clf,preds = train_miniRocket(X_train, y_train, X_test, y_test, dataset_name)
        elif predictor_name=="randomForest":
            clf, preds = train_randomForest(X_train, y_train, X_test, y_test, dataset_name)
        else:
            raise ValueError("predictor not found")

        predictor_dict[predictor_name] = {"clf": clf, "preds": preds}

    for segmentation_name in segmentation_names:

        init_segments = np.empty( (X_test.shape[0] , X_test.shape[1]), dtype=object) if X_test.shape[1] > 1  else (np.empty( X_test.shape[0] , dtype=object))
        results[dataset_name][segmentation_name] = {"segments": init_segments}
        for predictor_name in predictor_names:
            results[dataset_name][segmentation_name][predictor_name] = {'attributions': dict()}
        segmentation_method = segmentation_dict[segmentation_name]

        ts_list = []
        mask_list = []
        y_list = []

        for i in range(n_samples) : #
            
            # get current sample and label
            ts, y = X_test[i] , torch.tensor( y_test[i:i+1] )

            # get segment and its tensor representation
            current_segments = segmentation_method(ts)[:X_test.shape[1]]
            results[dataset_name][segmentation_name]['segments'][i] = current_segments
            mask = get_feature_mask(current_segments,ts.shape[-1])
            mask_list.append(mask)
            ts = torch.tensor(ts).repeat(1,1,1)	#TODO use something similar to np.expand_dim?
            ts_list.append(ts)
            y_list.append(y)

        for background_type in background_types:

            # background data
            if background_type=="zero":
                background_dataset = torch.zeros((1,) + X_train.shape[1:])
            elif background_type=="sampling":
                background_dataset = sample_background(X_train, n_background)
            elif background_type=="average":
                background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)

            for predictor_name in predictor_names:

                clf = predictor_dict[predictor_name]["clf"]
                results[dataset_name][segmentation_name][predictor_name]['y_test_pred'] = predictor_dict[predictor_name]["preds"]
                for background_type in background_types:
                    results[dataset_name][segmentation_name][predictor_name]['attributions'][background_type] = {"default": np.zeros( X_test.shape ,dtype=np.float32 )}

                with torch.no_grad():
                    
                    SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] \
                        else ShapleyValueSampling(forward_classification)

                    for i, (ts, mask, y) in enumerate(zip(ts_list, mask_list, y_list)):

                        # for sampling strategy repeat the ts many times as the background dataset size
                        ts = ts.repeat(background_dataset.shape[0],1,1) if background_type=="sampling" else ts
                        
                        # different call depending on predictor type
                        if predictor_name in predictors['scikit']:
                            # if using random forest flat everything
                            if predictor_name=="randomForest":
                                ts = ts.reshape( -1, n_chs*ts_length)
                                mask = mask.reshape( -1, n_chs*ts_length)
                                background_dataset = background_dataset.reshape( -1, n_chs*ts_length)
                            
                            tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)
                        
                        elif predictor_name in predictors['torch']:
                            # if use torch make sure everything is on selected device
                            ts = ts.to(device); y = y.to(device)
                            mask = mask.to(device) ; background_dataset =  background_dataset.to(device)
                            tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset)
                        
                        # 'un-flatten' for randomForest
                        if predictor_name=="randomForest":
                            tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])

                        # store current explanation in the data structure; if sampling store the mean
                        results[dataset_name][segmentation_name][predictor_name]['attributions'][background_type]["default"][i] = torch.mean(tmp, dim=0).cpu().numpy() if \
                            background_type=="sampling" else tmp[0].cpu().numpy()

                    if "normalized" in segmentation_types:
                        weights = np.array(list(map(lambda x: list(map(lambda y: lengths_to_weights(change_points_to_lengths(y, X_train.shape[-1])), x)), results[dataset_name][segmentation_name]["segments"])))
                        for background_type in background_types:
                            results[dataset_name][segmentation_name][predictor_name]['attributions'][background_type]["normalized"] = results[dataset_name][segmentation_name][predictor_name]['attributions'][background_type]["default"] * weights

        



training random forest
random forest accuracy is 1.0


In [22]:
results

{'average': {'default': array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0.,

In [ ]:
# dump result to disk
file_name = "all_results" # "_".join ( ( predictor_name, dataset_name ) )+".npy"
file_path = os.path.join("attributions", file_name)
np.save( file_path, results )

# OLD keep temporarily -  to be deleted

In [9]:


# # train model
# predictor_name = 'resNet'
# if predictor_name=='resNet':
# 	clf,preds = train_ResNet(X_train, y_train, X_test, y_test, dataset_name,device=device)
# elif predictor_name=='miniRocket':
# 	clf,preds = train_miniRocket(X_train, y_train, X_test, y_test, dataset_name)
# elif predictor_name=="randomForest":
# 	clf, preds = train_randomForest(X_train, y_train, X_test, y_test, dataset_name)
# else:
# 	raise ValueError(
# 		"predictor not found"
# 	)

# # segmentation
# segmentation_name = "nnsegment"
# segmentation_type = "normalized"
# segmentation_method = segmentation_dict[segmentation_name]

# # initialize data structure meant to contain the segments
# # TODO can I be cleaner here?
# segments =  np.empty( (X_test.shape[0] , X_test.shape[1]), dtype=object) if X_test.shape[1] > 1  else (
# 	np.empty( X_test.shape[0] , dtype=object))

# # create a dictionary to be dumped containing attribution and metadata
# results = {
# 	'attributions' : {},
# 	'segments' : segments,
# 	'y_test_true' : y_test,
# 	'y_test_pred' : preds,
# 	'label_mapping' : enc,
# }

# # result # dataset # model# segment # 

training ResNet
Epoch 1: train loss:  0.699, 	 train accuracy  0.440 
          test loss:  0.694,  	 test accuracy  0.493
Epoch 11: train loss:  0.688, 	 train accuracy  0.520 
          test loss:  0.691,  	 test accuracy  0.627
Epoch 21: train loss:  0.628, 	 train accuracy  0.640 
          test loss:  0.627,  	 test accuracy  0.680
Epoch 31: train loss:  0.518, 	 train accuracy  0.780 
          test loss:  0.588,  	 test accuracy  0.700
Epoch 41: train loss:  0.447, 	 train accuracy  0.800 
          test loss:  0.561,  	 test accuracy  0.707
Epoch 51: train loss:  0.489, 	 train accuracy  0.800 
          test loss:  0.568,  	 test accuracy  0.713
Epoch 61: train loss:  0.444, 	 train accuracy  0.800 
          test loss:  0.551,  	 test accuracy  0.720
Epoch 71: train loss:  0.442, 	 train accuracy  0.760 
          test loss:  0.551,  	 test accuracy  0.720
Epoch 81: train loss:  0.391, 	 train accuracy  0.820 
          test loss:  0.521,  	 test accuracy  0.767
Epoch 91: tra

In [10]:
# # define different background to be used and number of samples as n_background
# # TODO set a number of TOTAL sampling regardless of the background type?
# # zero, constant, average, multisample
# for bt in background_types:
# 	results['attributions'][bt] = np.zeros( X_test.shape ,dtype=np.float32 )

In [ ]:
# with torch.no_grad():
	
# 	SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] \
# 		else ShapleyValueSampling(forward_classification)
	
# 	for i in trange ( n_samples ) : #
		
# 		# get current sample and label
# 		ts, y = X_test[i] , torch.tensor( y_test[i:i+1] )

# 		# get segment and its tensor representation
# 		current_segments = segmentation_method(ts)[:X_test.shape[1]]
# 		results['segments'][i] = current_segments
# 		mask = get_feature_mask(current_segments,ts.shape[-1])

# 		ts = torch.tensor(ts).repeat(1,1,1)	#TODO use something similar to np.expand_dim?

# 		for background_type in background_types:

# 			# background data
# 			if background_type=="zero":
# 				background_dataset = torch.zeros((1,) + X_train.shape[1:])
# 			elif background_type=="sampling":
# 				background_dataset = sample_background(X_train, n_background)
# 			elif background_type=="average":
# 				background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)

# 			# for sampling strategy repeat the ts many times as the background dataset size
# 			ts = ts.repeat(background_dataset.shape[0],1,1) if background_type=="sampling" else ts

# 			# different call depending on predictor type
# 			if predictor_name in predictors['scikit']:
# 				# if using random forest flat everything
# 				if predictor_name=="randomForest":
# 					ts = ts.reshape( -1, n_chs*ts_length); mask = mask.reshape( -1, n_chs*ts_length);
# 					background_dataset = background_dataset.reshape( -1, n_chs*ts_length)
				
# 				tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)
			
# 			elif predictor_name in predictors['torch']:
# 				# if use torch make sure everything is on selected device
# 				ts = ts.to(device); y = y.to(device)
# 				mask = mask.to(device) ; background_dataset =  background_dataset.to(device)
# 				tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset)
			
# 			# 'un-flatten' for randomForest
# 			if predictor_name=="randomForest":
# 				tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])

# 			# store current explanation in the data structure; if sampling store the mean
# 			results['attributions'][background_type][i] = torch.mean(tmp, dim=0).cpu().numpy() if \
# 				background_type=="sampling" else tmp[0].cpu().numpy()

# 	if segmentation_type=="normalized":
# 		weights = np.array(list(map(lambda x: list(map(lambda y: lengths_to_weights(change_points_to_lengths(y, X_train.shape[-1])), x)), results["segments"])))
# 		for background_type in background_types:
# 			results["attributions"][background_type] *= weights

		


In [ ]:
# # dump result to disk
# file_name = "_".join ( ( predictor_name, dataset_name ) )+".npy"
# file_path = os.path.join("attributions", file_name)
# np.save( file_path, results )